# Rotulando Bens por sala

In [52]:
import pandas as pd
arquivo = 'serie_teste.xlsx'
tombos_df = pd.read_excel(arquivo)

autentica = pd.read_excel('ls.xlsx')
indisponivel = 'indisponivel'

login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

# display(tombos_df.head())

,tombo
0,38467
1,38469
2,25041
3,73203
4,12121212121


In [53]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [58]:
def rotular():
        
    abrir_suap()
 
    for tombo in tombos_df['tombo']:
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{tombo}/")
            nav.find_element_by_id('ajaxmultiselect_input_rotulos_add').click()
            nav.find_element_by_id('ajaxmultiselect_input_rotulos_add').send_keys("CAFI - SALA ESTUDOS BIBLIOTECA")
            time.sleep(2)
            nav.find_element_by_id('ajaxmultiselect_input_rotulos_add').send_keys(Keys.ENTER)
            nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[2]/input').click()
            time.sleep(2)
            
        except NoSuchElementException:
            continue
            
    nav.quit()
            

In [55]:
def remover_rotulo():
    
    abrir_suap()
    lista = []
    for tombo in tombos_df['tombo']:
        nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{tombo}/")
        rotulos = True
        while rotulos:
            try:
                nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{tombo}/")
                nav.find_element_by_class_name('ajaxmultiselect_remove').click()
                nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[2]/input').click()

            except NoSuchElementException:
                print('sem rotulos')
                rotulos = False
    nav.quit()

In [60]:
def listar_rotulos():
    
    abrir_suap()
        
    lista_tombo = []
    lista_carga = []
    lista_rotulo = []
    
    for tombo in tombos_df['tombo']:
        
        lista_tombo.append(tombo)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{tombo}/")

            carga = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[7]/td[2]").get_attribute('innerText')
#             print(carga)
            lista_carga.append(carga)
            
            rotulos = nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/fieldset/div[3]/div').get_attribute('innerText')
            rotulos = rotulos[10:]
            rotulos = rotulos.replace("\n ", '---')
            lista_rotulo.append(rotulos)
        
        except NoSuchElementException:
            lista_rotulo.append(indisponivel)
            lista_carga.append(indisponivel)
            continue
    
    df_rotulos = pd.DataFrame()
    df_rotulos['tombo'] = lista_tombo
    df_rotulos['carga'] = lista_carga
    df_rotulos['rotulo'] = lista_rotulo
#     display(df_rotulos)
    df_rotulos.to_excel("tabela_"+arquivo, index=False)
    
    nav.quit()